*** Step 1: 📦 Install Required Python Packages***

In [ ]:
!pip install flask flask-ngrok streamlit streamlit-ngrok joblib pandas scikit-learn matplotlib plotly reportlab spacy --quiet
!python -m spacy download en_core_web_sm


ERROR: Ignored the following versions that require a different python version: 0.55.2 Requires-Python <3.5
ERROR: Could not find a version that satisfies the requirement streamlit-ngrok (from versions: none)
ERROR: No matching distribution found for streamlit-ngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 95.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
pip install streamlit

  Using cached streamlit-1.47.1-py3-none-any.whl.metadata (9.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
pip install flask streamlit pandas scikit-learn requests

In [ ]:
!pip install flask-ngrok streamlit fpdf plotly requests --quiet
!pip install flask flask-ngrok streamlit pyngrok fpdf plotly



  Preparing metadata (setup.py) ... done


In [ ]:
!pip install flask flask-ngrok pyngrok streamlit colab_everything


In [ ]:
!pip install flask flask-ngrok pyngrok


In [ ]:
!pip install flask-ngrok
!pip install flask


In [ ]:
!pip install colab-everything pyngrok streamlit


In [ ]:
!pip install flask pyngrok
!pip install pyngrok


In [ ]:
!pip install streamlit pyngrok


In [ ]:
!pip install flask pyngrok pandas streamlit --quiet


In [ ]:
!pip install flask pyngrok pandas plotly reportlab streamlit --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.1 MB/s eta 0:00:00


In [ ]:
!python flask_app.py


python3: can't open file '/content/flask_app.py': [Errno 2] No such file or directory


In [ ]:
!kill $(lsof -t -i:5000)


kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


In [ ]:
!pkill ngrok || true

In [ ]:
# Run the modified Flask app
# This will output a public ngrok URL
# Make sure to update the FLASK_API variable in streamlit_app.py with this URL

In [ ]:
# Run the Streamlit app
# You can access the Streamlit app via the public ngrok URL printed after running the Flask app

*** Step 2: 📂 Upload Dataset File (Interactively)***

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ds_salaries.csv to ds_salaries (1).csv


*** Step 3: 🔐 Set Up ngrok Authentication***



In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("303pUHw4QeaPPT8ti0FBCo6sKhp_6R7CyNNKFPNK1h17M2yZk")

*** Step 4: 🔧 Import Libraries and Create Flask App***

In [ ]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
import pandas as pd
import threading

df = pd.read_csv("ds_salaries.csv")
app = Flask(__name__)

@app.route('/')
def home():
    return "✅ Flask backend is running"

@app.route('/predict', methods=['POST'])
def predict():
    if df is None:
        return jsonify({'error': 'Dataset not loaded'}), 500
    data = request.get_json()
    prediction = df['salary_in_usd'].mean()
    return jsonify({'predicted_salary': round(prediction, 2)})

# 🔄 Start Flask on a new port
def run_flask():
    app.run(port=5002)

threading.Thread(target=run_flask).start()

# 🌍 Create ngrok tunnel on same port
public_url = ngrok.connect(5002)
print("🔗 Public ngrok URL:", public_url)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5002
INFO:werkzeug:Press CTRL+C to quit


🔗 Public ngrok URL: NgrokTunnel: "https://bb14c946fcf9.ngrok-free.app" -> "http://localhost:5002"


*** Step 5: 📺 (Commented Out) Streamlit App Code***


In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import requests
import time
import plotly.graph_objects as go
from fpdf import FPDF

# Replace with your actual Flask ngrok public URL
FLASK_API = "https://xxxxxx.ngrok.io/predict"

st.set_page_config(page_title="SalarIQ", page_icon="💼", layout="wide")

# 🎯 Title + Slogan (Centered)
st.markdown("""
<div style="text-align: center;">
    <h1>💼 SalarIQ</h1>
    <h3>🚀 Unlock Your True Earning Potential</h3>
    <p><em>AI-powered insights to elevate your career journey.</em></p>
</div>
""", unsafe_allow_html=True)
st.markdown("---")

# 🔍 Input Form
with st.form("predict_form"):
    col1, col2 = st.columns(2)
    with col1:
        experience = st.selectbox("Experience Level", ["Entry", "Mid", "Senior", "Executive"])
        employment = st.selectbox("Employment Type", ["Full-time", "Part-time", "Contract", "Internship"])
        job_title = st.text_input("Job Title")
        residence = st.text_input("Employee Residence")
    with col2:
        remote_ratio = st.slider("Remote Ratio (%)", 0, 100, 50)
        company_location = st.text_input("Company Location")
        company_size = st.selectbox("Company Size", ["Small", "Medium", "Large"])
        resume = st.file_uploader("📄 Upload Resume (PDF/CSV)", type=["pdf", "csv"])
    submit = st.form_submit_button("🔍 Predict & Analyze")

# 💡 Prediction & Analysis
if submit:
    with st.spinner("🔍 Predicting and analyzing your profile..."):
        features = [1, 2, 5, 3, remote_ratio, 2, 3]  # Dummy encoding (replace with your logic)
        salary = None

        try:
            response = requests.post(FLASK_API, json={"features": features})
            salary = response.json().get('predicted_salary')
        except:
            st.warning("⚠️ Prediction service is offline. Showing estimated insights.")

        if salary:
            st.success(f"💰 Predicted Salary: ${salary}")
            st.balloons()
        else:
            salary = 70000
            st.info(f"💰 Estimated Salary: ${salary} (default)")

        # 📈 Salary Growth Chart
        years = list(range(1, 6))
        projected = [round(salary * (1.05 ** y)) for y in years]
        fig = go.Figure(data=go.Scatter(x=years, y=projected, mode='lines+markers'))
        fig.update_layout(title="📈 Salary Growth Over 5 Years", xaxis_title="Years", yaxis_title="USD")
        st.plotly_chart(fig)

        # 🧠 Career Suggestions
        st.markdown("### 🧠 Career Suggestions Based on Your Profile")
        st.info("You're a strong match for: **ML Engineer**, **Data Analyst**, **BI Developer**")

        # 🌍 Country Comparison
        st.markdown("### 🌍 Top 5 Country Comparison")
        countries = ["US", "Germany", "India", "UK", "Canada"]
        salaries = [salary * 1.2, salary * 1.15, salary * 0.8, salary, salary * 1.05]
        country_fig = go.Figure([go.Bar(x=countries, y=salaries, marker_color='skyblue')])
        country_fig.update_layout(title="Top 5 Country Salary Comparison")
        st.plotly_chart(country_fig)

        # 📝 Resume Upload
        if resume:
            st.markdown("### Resume Parsing Feedback")
            st.warning("🚧 Resume parsing and NLP analysis coming soon!")

        # 📄 Generate PDF Report
        st.markdown("### 📄 Download Your Report")

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=14)
        pdf.cell(200, 10, txt="SalarIQ Salary Report", ln=True, align='C')
        pdf.ln(10)
        pdf.set_font("Arial", size=12)
        pdf.cell(200, 10, txt=f"Predicted Salary: ${salary}", ln=True)

        pdf.cell(200, 10, txt="Salary Projection Over 5 Years:", ln=True)
        for i, val in enumerate(projected):
            pdf.cell(200, 10, txt=f"Year {i+1}: ${val}", ln=True)

        pdf.ln(5)
        pdf.cell(200, 10, txt="Top 5 Country Salary Comparison:", ln=True)
        for i in range(len(countries)):
            pdf.cell(200, 10, txt=f"{countries[i]}: ${int(salaries[i])}", ln=True)

        pdf.ln(5)
        pdf.cell(200, 10, txt="Suggested Roles: ML Engineer, Data Analyst, BI Developer", ln=True)

        pdf.output("report.pdf")

        with open("report.pdf", "rb") as f:
            st.download_button("📥 Download PDF Report", f, file_name="salary_report.pdf", mime="application/pdf")


Overwriting streamlit_app.py


**Step 6: ▶️ Run Streamlit App**

In [ ]:
!nohup streamlit run streamlit_app.py --server.port 8501 &



nohup: appending output to 'nohup.out'


**Step 7: 🌐 Create Public ngrok URL for Flask App**

In [ ]:
public_url = ngrok.connect(8501)
print("🔗 Public URL:", public_url)

🔗 Public URL: NgrokTunnel: "https://b1ca6e495b80.ngrok-free.app" -> "http://localhost:8501"
